In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

In [2]:
import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# We import the simulation initial conditions and atmospheric density model
The files needed are:
* **debris_simulation_ic.pk** - created by the notebook 1
* **best_fit_density.pk** - created by the notebook 1b

In [3]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,collision_radius,to_satcat,satcat,debris = pkl.load(file)

* **r**: contains the initial position of all satellites to be simulated (SI units)
* **v**: contains the initial velocity of all satellites to be simulated (SI units)
* **radius**: contains all the estimated radii for the various objects (in meters)
* **to_satcat**: contains the indexes in the satcat of the corresponding r,v,radius entry
* **satcat**: the satcat
* **debris**: the corresponding pykep planets

In [4]:
with open("data/best_fit_density.pk", "rb") as file:
    best_x = pkl.load(file)

In [5]:
# We need to create an array containing all B*
BSTARS = []
for idx in to_satcat:
    BSTARS.append(float(satcat[idx]["BSTAR"]))
# We put the BSTAR in SI units
BSTARS = np.array(BSTARS) / pk.EARTH_RADIUS
# We remove negative BSTARS setting the value to zero in those occasions
BSTARS[BSTARS<0] = 0.

# We build the dynamical system to integrate

In [6]:
dyn = csc.dynamics.simple_earth(J2=True, J3=False, C22S22=True, sun=False, moon=False, SRP=False, drag=True)

# We setup the simulation

In [7]:
csc.set_logger_level_trace()

In [8]:
def remove_particle(idx, r_ic, v_ic, BSTARS,to_satcat, collision_radius):
    r_ic = np.delete(r_ic, idx, axis=0)
    BSTARS = np.delete(BSTARS, idx, axis=0)
    v_ic = np.delete(v_ic, idx, axis=0)
    to_satcat = np.delete(to_satcat, idx, axis=0)
    collision_radius = np.delete(collision_radius, idx, axis=0)
    return r_ic, v_ic, BSTARS, to_satcat, collision_radius

In [9]:
# Before starting we need to remove all particles inside our playing field
min_radius = pk.EARTH_RADIUS+150000.
inside_the_radius = np.where(np.linalg.norm(r_ic,axis=1) < min_radius)[0]
print("Removing orbiting objects:")
for idx in inside_the_radius:
    print(satcat[to_satcat[idx]]["OBJECT_NAME"], "-", satcat[to_satcat[idx]]["OBJECT_ID"])
r_ic, v_ic, BSTARS,to_satcat, collision_radius = remove_particle(inside_the_radius, r_ic, v_ic, BSTARS,to_satcat, collision_radius)

Removing orbiting objects:
LEMUR 2 ROCKETJONAH - 2017-071E
ISARA - 2017-071P
FREGAT DEB - 2011-037EM
STARLINK-1684 - 2020-070H
COSMOS 1408 DEB - 1982-092Z
COSMOS 1408 DEB - 1982-092AK
COSMOS 1408 DEB - 1982-092ES
COSMOS 1408 DEB - 1982-092FK
COSMOS 1408 DEB - 1982-092FY
COSMOS 1408 DEB - 1982-092GU
COSMOS 1408 DEB - 1982-092NA
COSMOS 1408 DEB - 1982-092PV
COSMOS 1408 DEB - 1982-092PW
COSMOS 1408 DEB - 1982-092RM
COSMOS 1408 DEB - 1982-092ACG
COSMOS 1408 DEB - 1982-092AQC
COSMOS 1408 DEB - 1982-092ARK
COSMOS 1408 DEB - 1982-092AXA
COSMOS 1408 DEB - 1982-092AXD
COSMOS 1408 DEB - 1982-092BDB
COSMOS 1408 DEB - 1982-092BFU
COSMOS 1408 DEB - 1982-092BKD


In [10]:
# Prepare the data in the shape expected by the simulation object.
ic_state = np.hstack([r_ic, v_ic, collision_radius.reshape((r_ic.shape[0], 1))])
pars = BSTARS.reshape((r_ic.shape[0], 1))

# Uncommen these lines to save the txt files to be used in the cpp simulations
#np.savetxt("test_ic_19647.txt", ic_state.reshape((-1, 1))
#np.savetxt("test_par_19647.txt", pars)

In [11]:
sim = csc.sim(ic_state,0.23 * 806.81,dyn=dyn,pars=pars, reentry_radius=min_radius)

# We run the simulation

In [12]:
# new_state  =deepcopy(ic_state)
# new_pars  =deepcopy(pars)
# new_to_satcat  =deepcopy(to_satcat)

rng = np.random


In [13]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], pk.MU_EARTH)

            sim.state[idx,0:3] = r
            sim.state[idx,3:6] = v
            sim.state[idx,6] = size
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log on screen 
        print(satcat[to_satcat[pi]]["OBJECT_NAME"].strip() + ", " + satcat[to_satcat[pi]]["OBJECT_ID"].strip() + ", ", sim.time*pk.SEC2DAY, "REMOVED")
        # We remove the re-entered object and restart the simulation
        new_state = np.delete(sim.state,pi,axis=0)
        new_pars = np.delete(sim.pars,pi,axis=0)
        
        sim.set_new_state(new_state)
        sim.pars[:] = new_pars
        
        # new_r_ic = np.vstack((sim.x,sim.y,sim.z)).transpose()
        # new_v_ic = np.vstack((sim.vx,sim.vy,sim.vz)).transpose()
        # new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius = remove_particle(pi, new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius)
        # sim.set_new_state(new_r_ic[:,0],new_r_ic[:,1],new_r_ic[:,2],new_v_ic[:,0],new_v_ic[:,1],new_v_ic[:,2],new_reentry_radius, pars=[new_BSTARS])
pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

COSMOS 1408 DEB, 1982-092FH,  0.0011018074694589025 REMOVED


AttributeError: 'cascade.core.sim' object has no attribute 'set_new_state'

In [ ]:
with open("data/deterministic_simulation.pk", "wb") as file:
    pkl.dump((r_ic, v_ic, BSTARS, to_satcat, collision_radius), file)